In [1]:
!git clone https://github.com/evgmaslov/structure_generation

Cloning into 'structure_generation'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 14 (delta 0), reused 7 (delta 0), pack-reused 0
Receiving objects: 100% (14/14), 910.60 KiB | 12.30 MiB/s, done.


In [11]:
from pathlib import Path
import sys
repo = Path("/content/structure_generation").resolve()
sys.path.append("/content/structure_generation/")

In [3]:
!cd {repo} && pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 24.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.w

In [4]:
import numpy as np
import torch
from torch import nn
import math
from typing import Optional, Tuple, Union, List
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import random
import os
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
import shutil
from diffusers import DDPMScheduler, UNet3DConditionModel, UNet2DModel, DDPMPipeline
from diffusers.optimization import get_cosine_schedule_with_warmup
from accelerate import Accelerator

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

## Downloading sources, if you use kaggle

In [ ]:
!conda install -y gdown

In [ ]:
import gdown
url = 'https://drive.google.com/uc?export=download&id=1hDhDttFpZ_-TDVo_-yBzhafh75mvis6L'
data_path = '/kaggle/working/dataset.zip'
gdown.download(url, data_path)

## Downloading sources, if you use colab

In [5]:
from google.colab import drive
drive.mount('/content/drive')
data_path = "/content/drive/MyDrive/structure_generation/dataset.zip"

Mounted at /content/drive


## Training

In [7]:
!unzip {data_path}

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: dataset/n10-id5499.tesr  
  inflating: dataset/n10-id55.tesr   
  inflating: dataset/n10-id550.tesr  
  inflating: dataset/n10-id5500.tesr  
  inflating: dataset/n10-id5501.tesr  
  inflating: dataset/n10-id5502.tesr  
  inflating: dataset/n10-id5503.tesr  
  inflating: dataset/n10-id5504.tesr  
  inflating: dataset/n10-id5505.tesr  
  inflating: dataset/n10-id5506.tesr  
  inflating: dataset/n10-id5507.tesr  
  inflating: dataset/n10-id5508.tesr  
  inflating: dataset/n10-id5509.tesr  
  inflating: dataset/n10-id551.tesr  
  inflating: dataset/n10-id5510.tesr  
  inflating: dataset/n10-id5511.tesr  
  inflating: dataset/n10-id5512.tesr  
  inflating: dataset/n10-id5513.tesr  
  inflating: dataset/n10-id5514.tesr  
  inflating: dataset/n10-id5515.tesr  
  inflating: dataset/n10-id5516.tesr  
  inflating: dataset/n10-id5517.tesr  
  inflating: dataset/n10-id5518.tesr  
  inflating: dataset/n10-id5519.tesr  

In [12]:
from data import TesselationDataset, collate_fn
from models import DiffusersDDPM3D
from utils import Trainer

In [13]:
train_names, val_names = train_test_split(os.listdir("/content/dataset")[:1000], test_size=0.1)

In [14]:
batch_size = 2
train_data_params = {
    "tess_dir":"/content/dataset",
    "names":train_names,
    "dims":3
}
train_dataset = TesselationDataset(train_data_params)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=lambda x: collate_fn(x, train_dataset))

val_data_params = {
    "tess_dir":"/content/dataset",
    "names":val_names,
    "dims":3
}
val_dataset = TesselationDataset(val_data_params)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=lambda x: collate_fn(x, val_dataset))

trainer_args = {
    "train_dataloader":train_dataloader,
    "val_dataloader":val_dataloader,
    "score":None,
    "train_score_computing_frequency":1,
    "optimizer":torch.optim.AdamW,
    "optimizer_args":{"lr":3e-4},
    "scheduler":get_cosine_schedule_with_warmup,
    "scheduler_args":{
        "num_warmup_steps":100,
        "num_training_steps":(len(train_dataloader) * 2)
    },
    "freezer":None,
    "freezer_args":None,
    "training_controller":None,
    "n_epochs":50,
    "device":"cuda" if torch.cuda.is_available() else "cpu",
    "backup_path":"/content/drive/MyDrive/structure_generation/model_difusers3d_v3.pt",
    "show_outputs":True,
    "show_outputs_every":2,
    "n_outputs":1,
    "is_saving_history":False,
}

trainer = Trainer(trainer_args)

In [15]:
model_args = {
    "device":trainer.device,
    "sample_shape": (32, 32, 32),
    "noise_scheduler": DDPMScheduler(num_train_timesteps=1000),
    "unet_n_blocks": 2,
    "unet_channels": 64,
    "is_attn":(False, False, True, True),
    "ch_mults":(1, 2, 2, 4),
    "channels":(64, 128, 128, 256),
}
model = DiffusersDDPM3D(model_args)

In [16]:
torch.cuda.empty_cache()

In [ ]:
trainer.train(model)